# Foramlising rules with Kanren for Python

Below are a couple of examples with one rule each of using Kanren for writing facts and rules and querying those. Using those examples as inspiration, write down the code to implement the rules you have for choosing a type of dog to adopt. Also include the description of a person/household and see what is being inferred from your rule.

You might want to think about the kind of interface you could develop on top of your rule-based system to ask the relevant questions to the user and help them decide on the kind of dogs to adopt.

**Note:** This is a colab notebook in read-only mode. Don't forget to save a copy before modifying it or you might lose your changes.

## Installantion of Kanren and examples


In [2]:
pip install minikanren


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\belen\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


This first example is very simple: It only includes one rule, and manipulates only string.

In [3]:
from kanren import Relation, facts, var, run, lall, conde
parent = Relation()
facts(parent, ("Homer", "Bart"),
              ("Homer", "Lisa"),
              ("Abe",  "Homer"))
x = var()
print("Parent of Bart:", run(1, x, parent(x, "Bart")))
print("Children of Homer:", run(2, x, parent("Homer", x)))

def grandparent(x, z):
    y = var()
    return lall(parent(x, y), parent(y, z))

print("Grandparent of Bart:", run(1, x, grandparent(x, 'Bart')))

Parent of Bart: ('Homer',)
Children of Homer: ('Bart', 'Lisa')
Grandparent of Bart: ('Abe',)


This example also only has one rule, but manipulate more complex objects in the form of instances of classes.

In [5]:
studies = Relation()
teaches = Relation()

class Person:
  def __init__(self, name):
    self.name = name
  def __str__(self):
    return self.name

class Course:
  def __init__(self, code, topic):
    self.code = code
    self.topic = topic
  def __str__(self):
    return self.code+":"+self.topic

charlie=Person("Charlie")
olivia=Person("Olivia")
jack=Person("Jack")
arthur=Person("Arthur")
kirke = Person("Kirke")
collins = Person("Collins")
juniper = Person("Juniper")

csc135 = Course("csc135", "Python Programming")
csc131 = Course("csc131", "Computer Architectures")
csc134 = Course("csc134", "Web Development")
csc171 = Course("csc171", "Java")

facts(studies, (charlie, csc135), # charlie studies csc135
               (olivia, csc135),  # olivia studies csc135
               (jack, csc131),    # jack studies csc131
               (arthur, csc134))  # arthur studies csc134

facts(teaches, (kirke, csc135),   # kirke teaches csc135
               (collins, csc131), # collins teaches csc131
               (collins, csc171), # collins teaches csc171
               (juniper, csc134)) # juniper teaches csc134

def professor(x, y): #A rule (!)
  c = var() #We don't care about what course here. There's no need to declare it, it will scann it automatically
  return conde((teaches(x, c), studies(y, c))) #both of this facts to hold in order to this rule to be true

what = var()
charlieStu = run(0, what, studies(charlie, what))
print("Charlie studies", charlieStu[0])

students = var()
kirkeStu = run(0, students, professor(kirke, students))
print("The students of Kirke are", end=" ")
for s in kirkeStu:
  print(s, end=" ")
print()

Charlie studies csc135:Python Programming
The students of Kirke are Charlie Olivia 


## Your knowledge base

Write your code below and test it.

Note that in rules, the conde function can be used to express:

logical "and": conde( (A,B) ) means A and B
logical "or": conde( [A], [B] ) means A or B

The best way to write rules is to several simple ones.

It is best to start representing the objects as strings rather than through classes, as it will make it easier to start with.

I will take into consideration two elements of lifestyle to recommend a group of possibles breeds to a person:
1. Level of daily activity
2. If they have or they don't have children

In [15]:
### Rules for level of activity ### 
adopts = Relation ()
active = Relation()


# Object orientated approach. Class creations first.
class breed:
    def __init__(self, name, activity_level):
        self.name = name
        self.activity_level = activity_level
    def __str__(self):
        return self.name

# Lifestyle characteristics of a potential adopter.

class adopter:
    def __init__(self, activity_level, children): #Children will be boolean
        self.activity_level = activity_level
        self.children = children
    def __str__(self):
        return self.activity_level, self.children

# Create instances of the classes to state facts
weimaraner = breed('Weimaraner', 'High')
beagle = breed('Beagle', 'Low')
myself = adopter('High', False)
hanady = adopter('Low', True)

# Stating facts ? 

facts(active(myself, weimaraner))

def potential_adopter(p, d): #A rule (!)
  a = var() #We don't care about what course here. There's no need to declare it, it will scann it automatically
  return conde((active(p, a), active(a, d))) #both of this facts to hold in order to this rule to be true

which = var()
recommendation = run(0, which, active(myself, which))
print("You should adopt", recommendation)

You should adopt ()


In [26]:
class dog:
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return self.name 

class person:
    def __init__(self,name):
        self.name = name
    def __str__(self):
        return self.name

dalmatian = dog('Dalmatian')
me = person('Belen')

# ------- FACTS ------------

activity_levels_dog = Relation ()
activity_levels_person = Relation() #Each object needs one relation of its own

facts(activity_levels_dog, (dalmatian, 'High energy'))
facts(activity_levels_person,(me, 'High energy'))

def potential_adopter(p, d): #A rule (!)
  a = var() #We don't care about what course here. There's no need to declare it, it will scann it automatically
  return conde((activity_levels_dog(d, a), activity_levels_person(p, a))) #both of this facts to hold in order to this rule to be true

which = var()
recommendation = run(0, which, potential_adopter(me, which))
print("You should adopt", recommendation[0])

You should adopt Dalmatian


In [ ]:
class house:
    def __init__(self, size, name):
        self.size = size
        self.name = name
    def __str__(self):
        return self.size, self.name

class dog_breed:
    def __init__(self, size, type):
        self.size = size
        self.type = type
    def __str__(self):
        return self.size, self.name
    
size_dog = Relation()
size_house = Relation()
adopts = Relation()

house_a = house('big', 'a')
house_b = house('small', 'b')
dog_a = dog_breed('big', 'German sheperd')
dog_b = dog_breed('small', 'chihuahua')

facts(adopts, (house_a, dog_a))


